In [ ]:
import numpy as np
import msmrd2
import random
from msmrd2.potentials import patchyParticleAngular
from msmrd2.integrators import overdampedLangevin as odLangevin
import msmrd2.tools.quaternions as quaternions
from msmrd2.markovModels import continuousTimeMarkovStateModel as ctmsm
from msmrd2.markovModels import msmrdMarkovStateModel as msmrdMSM
from msmrd2.integrators import msmrdIntegrator
import multiprocessing
from multiprocessing import Pool
import pickle

## TEST Bechmark

In [ ]:
'''
Creates an MD simulation of two particle and calculates their first passage times (FPTs) from a random 
bound state to any unbound state. The data is written to '../data/dimer/first_passage_times/filename_here.
'''

# Main parameters for particle and integrator
numparticles = 2
dt = 0.0001 #0.00001 #0.000005
bodytype = 'rigidbody'
D = 1.0
Drot = 1.0
initialState = 'B'
radialBounds = [1.4, 2.2]
relativeDistanceCutOff = radialBounds[1]
# Other important parameters
boxsize = 6

# Parameters of patchy Particle potential with angular dependence (make sure it is consistent with msmrd data)
sigma = 1.0
strength = 50.0
angularStrength = 10.0
angleDiff = 3*np.pi/5.0
patch1 = np.array([np.cos(angleDiff/2),np.sin(angleDiff/2),0.])
patch2 = np.array([np.cos(-angleDiff/2),np.sin(-angleDiff/2),0.])
patchesCoordinates = [patch1, patch2]

# Bound states, needed to calculate boundstate of patchydimer
boundStatesA = [1, 2, 5, 6] # U-shaped bound dimer, corresponds to A state
boundStatesB = [3, 4, 7, 8] # Zigzag-shaped bound dimer, corresponds to B state

In [ ]:
def generateParticleList(state, boxsize, D, Drot, randomSeed = -1):
    '''
    Generate a random particle list of two particles corresponding to either state A or state B. As each
    state has several different configurations, this functions picks one randomly. The definition of the
    states is taken form patchyDimer.cpp
    :param state: bound state in which the two particle list should be in, A or B
    :param D and Drot: diffusion coefficients of particles.
    :param randomSeed: seed for python random generator. Important to specify in parallel runs. Default value of -1
    will use the default seed.
    :return: random particle list corresponding to either state A or state B.
    '''

    # Transform boxsize to vector if neccesarry.
    boxsize = boxsize - 1 # to ensure the whole compund is initially inside box (1 is the norm of relpos below)
    if np.isscalar(boxsize):
        boxsize = np.array([boxsize, boxsize, boxsize])

    position1 = np.array([0.0, 0.0, 0.0])
    #position1 = np.array([boxsize[0]*random.random()-0.5*boxsize[0],
    #                      boxsize[1]*random.random()-0.5*boxsize[1],
    #                      boxsize[2]*random.random()-0.5*boxsize[2]])
    orientation1 = np.array([1.0, 0.0, 0.0, 0.0])
    # Define relative position
    relpos1 = np.array([np.cos(angleDiff / 2.0),np.sin(angleDiff / 2.0), 0])
    relpos2 = np.array([np.cos(angleDiff / 2.0),np.sin(-1*angleDiff / 2.0), 0])
    relPos1orthogonal = np.array([-1.0 * np.sin(angleDiff / 2.0), np.cos(angleDiff / 2.0), 0.0])
    relPos2orthogonal = np.array([np.sin(angleDiff / 2.0), np.cos(angleDiff / 2.0), 0.0])
    # Define relative rotations
    rotations = [None]*8
    rotations[0] = np.pi * relPos1orthogonal
    rotations[1] = np.array([0.0, 0.0, -2 * np.pi / 5.0])
    rotations[2] = np.array([0.0, 0.0, np.pi])
    rotations[3] = np.array([0.0, np.pi, 0.0])
    # --first 4 rotations correspond to binding on top patch of particle 1, next 4 rotations to bottom patch
    rotations[4] = np.pi * relPos2orthogonal
    rotations[5] = np.array([0.0, 0.0, 2 * np.pi / 5.0])
    rotations[6] = np.array([0.0, 0.0, np.pi])
    rotations[7] = np.array([0.0, np.pi, 0.0])
    # Convert axis-angle rotations to quaternions
    quatRotations = [None]*8
    for i in range(8):
        quatRotations[i] = quaternions.angle2quat(rotations[i])
    # Assign correct position2 depending on the states
    pos2 = [None]*8
    pos2[0:4] = [position1 + relpos1]*4
    pos2[4:8] = [position1 + relpos2]*4

    # Assign position and orientation depending on initial state A or B
    if state == 'A':
        substate = random.choice([1,2,5,6])
        position2 = pos2[substate - 1]
        orientation2 = quatRotations[substate - 1]
    if state == 'B':
        substate = random.choice([3,4,7,8])
        position2 = pos2[substate - 1]
        orientation2 = quatRotations[substate - 1]
    part1 = msmrd2.particle(D, Drot, position1, orientation1)
    part2 = msmrd2.particle(D, Drot, position2, orientation2)
    partlist = msmrd2.integrators.particleList([part1, part2])
    return partlist

In [ ]:
# Define dummy trajectory to extract bound states from python (needed to use getState function)
dummyTraj = msmrd2.trajectories.patchyDimer(2,1)

# Define simulation boundaries (choose either spherical or box)
boxBoundary = msmrd2.box(boxsize, boxsize, boxsize, 'periodic')

# Define potential
potentialPatchyParticleAngular = patchyParticleAngular(sigma, strength, angularStrength, patchesCoordinates)

# Define integrator and boundary (over-damped Langevin)
seed = int(-1) # Negative seed, uses random device as seed
integrator = odLangevin(dt, seed, bodytype)
integrator.setBoundary(boxBoundary)
integrator.setPairPotential(potentialPatchyParticleAngular)

# Generate random position and orientation particle list with two particles
partlist = generateParticleList(initialState, boxsize, D, Drot)

# Calculates the first passage times for a given bound state. Each trajectory is integrated until
# a bound state is reached. The output in the files is the elapsed time.
bound = True
i=0
while(bound):
    integrator.integrate(partlist)
    boundState = dummyTraj.getState(partlist[0], partlist[1])
    print(boundState)
    i = i + 1
    if boundState == 0:
        bound = False
        print(initialState, integrator.clock, i)
        break
    elif integrator.clock >= 2000.0:
        bound = False
        Print('Failed at:', integrator.clock)
        break

## TEST MSMRD

In [ ]:
# Main parameters for particle and integrator
partTypes = 0
numBoundStates = 8
maxNumBoundStates = 10
numParticleTypes = 1 # num. of particle types (not states) in unbound state
# Other important parameters
lagtime = 600
boxsize = 6

# Discretization parameters (need to be consistent with the on used to generate the rate dictionary
numSphericalSectionsPos = 7
numRadialSectionsQuat = 5
numSphericalSectionsQuat = 7
totalnumSecsQuat = numSphericalSectionsQuat*(numRadialSectionsQuat -1) + 1
numTransitionsStates = numSphericalSectionsPos * totalnumSecsQuat #203

# Parameters to define continuous-time MSM for unbound dynamics: unboundMSM (assumed same for all particles)
MSMtype = 0
ratematrix = np.array([[0]]) # no unbound dynamics
Dlist = np.array([1.0])
Drotlist = np.array([1.0])

# Parameters to define coupling Markov model for bound dynamics: couplingMSM
Dbound = 0.5*np.ones(numBoundStates)
DboundRot = np.ones(numBoundStates)


In [ ]:
def generateParticleListMSMRD(state, boxsize, types, unboundMSMs, randomSeed = -1):
    '''
    Generate a random particle list of two particles corresponding to either state A or state B. As each
    state has several different configurations, this functions picks one randomly. The definition of the
    states is taken form patchyDimer.cpp
    :param state: bound state in which the two particle list should be in, A or B
    :param boxsize: size of simulation box, if scalar it assumes the three box edges are the same in all dimensions
    :param unboundMSMs: list of unboundMSM, needed to extract diffusion coefficients of particles.
    :param randomSeed: seed for python random generator. Important to specify in parallel runs. Default value of -1
    will use the default seed.
    :return: random particle list corresponding to either state A or state B.
    '''

    # Transform boxsize and type to vector if neccesary.
    boxsize = boxsize - 1 # to ensure the whole compund is initially inside box (1 is the norm of relpos below)
    if np.isscalar(boxsize):
        boxsize = np.array([boxsize, boxsize, boxsize])

    if np.isscalar(types):
        newtypes = np.ones(2, dtype = 'int')
        types = (types*newtypes).astype(int)

    # Obtain diffusion coefficients from unboundMSM
    D1 = unboundMSMs[types[0]].D[0]
    Drot1 = unboundMSMs[types[0]].Drot[0]
    D2 = unboundMSMs[types[1]].D[0]
    Drot2 = unboundMSMs[types[1]].Drot[0]


    position1 = np.array([0.0, 0.0, 0.0])
    #position1 = np.array([boxsize[0]*random.random()-0.5*boxsize[0],
    #                    boxsize[1]*random.random()-0.5*boxsize[1],
    #                     boxsize[2]*random.random()-0.5*boxsize[2]])
    orientation1 = np.array([1.0, 0.0, 0.0, 0.0])
    # Define relative position
    relpos1 = np.array([np.cos(angleDiff / 2.0),np.sin(angleDiff / 2.0), 0])
    relpos2 = np.array([np.cos(angleDiff / 2.0),np.sin(-1*angleDiff / 2.0), 0])
    relPos1orthogonal = np.array([-1.0 * np.sin(angleDiff / 2.0), np.cos(angleDiff / 2.0), 0.0])
    relPos2orthogonal = np.array([np.sin(angleDiff / 2.0), np.cos(angleDiff / 2.0), 0.0])
    # Define relative rotations
    rotations = [None]*8
    rotations[0] = np.pi * relPos1orthogonal
    rotations[1] = np.array([0.0, 0.0, -2 * np.pi / 5.0])
    rotations[2] = np.array([0.0, 0.0, np.pi])
    rotations[3] = np.array([0.0, np.pi, 0.0])
    # --first 4 rotations correspond to binding on top patch of particle 1, next 4 rotations to bottom patch
    rotations[4] = np.pi * relPos2orthogonal
    rotations[5] = np.array([0.0, 0.0, 2 * np.pi / 5.0])
    rotations[6] = np.array([0.0, 0.0, np.pi])
    rotations[7] = np.array([0.0, np.pi, 0.0])
    # Convert axis-angle rotations to quaternions
    quatRotations = [None]*8
    for i in range(8):
        quatRotations[i] = quaternions.angle2quat(rotations[i])
    # Assign correct position2 depending on the states
    pos2 = [None]*8
    pos2[0:4] = [position1 + relpos1]*4
    pos2[4:8] = [position1 + relpos2]*4

    # Assign position and orientation depending on initial state A or B
    substate = 0
    if state == 'A':
        substate = random.choice([1,2,5,6])
        position2 = pos2[substate - 1]
        orientation2 = quatRotations[substate - 1]
    if state == 'B':
        substate = random.choice([3,4,7,8])
        position2 = pos2[substate - 1]
        orientation2 = quatRotations[substate - 1]
    part1 = msmrd2.particle(types[0], substate, D1, Drot1, position1, orientation1)
    part2 = msmrd2.particle(types[1], substate, D2, Drot2, position2, orientation2)
    part1.setBoundTo(1)
    part2.setBoundTo(0)
    part1.deactivateMSM()
    part2.deactivateMSM()
    part2.deactivate()
    part2.setPosition([10000000.0, 10000000.0, 10000000.0])
    partlist = msmrd2.integrators.particleList([part1, part2])
    return partlist

In [ ]:
# Define dummy trajectory to extract bound states from python (needed to use getState function)
dummyTraj = msmrd2.trajectories.patchyDimer(2,1)

# Define discretization
discretization = msmrd2.discretizations.positionOrientationPartition(relativeDistanceCutOff,
                                        numSphericalSectionsPos, numRadialSectionsQuat, numSphericalSectionsQuat)

# Define boundary
boxBoundary = msmrd2.box(boxsize,boxsize,boxsize,'periodic')

# Load rate dicitionary
pickle_in = open("../../data/pickled_data/ratedictionary_dimer_t4.00E+06_s25_lagt" + str(lagtime)
                 +  ".pickle","rb")
rateDictionary = pickle.load(pickle_in)

# Set unbound MSM
seed = -int(1) # Negative seed, uses random device as seed
unboundMSM = ctmsm(MSMtype, ratematrix, seed)
unboundMSM.setD(Dlist)
unboundMSM.setDrot(Drotlist)

# Set coupling MSM
seed = -int(1) # Negative seed, uses random device as seed
couplingMSM = msmrdMSM(numBoundStates, numTransitionsStates, seed, rateDictionary)
couplingMSM.setDbound(Dbound, DboundRot)
couplingMSM.setMaxNumberBoundStates(maxNumBoundStates)

# Define integrator, boundary and discretization
seed = -int(1) # Negative seed, uses random device as seed
integrator = msmrdIntegrator(dt, seed, bodytype, numParticleTypes, radialBounds, unboundMSM, couplingMSM)
integrator.setBoundary(boxBoundary)
integrator.setDiscretization(discretization)

# Generate random position and orientation particle list with two particles
partlist = generateParticleListMSMRD(initialState, boxsize, partTypes, [unboundMSM])

# Calculates the first passage times for a given bound state. Each trajectory is integrated until
# a bound state is reached. The output in the files is the elapsed time.
bound = True
i=0
while(bound):
    integrator.integrate(partlist)
    currentState = partlist[0].state
    position1 = partlist[1].position
    transitionState = dummyTraj.sampleDiscreteState2(partlist[0], partlist[1]) # only valid if unbound
    print(currentState, transitionState, position1)
    i = i + 1
    if (transitionState == 0) and (currentState == 0):
        bound = False
        print(initialState, integrator.clock, i)
        break
    elif integrator.clock >= 2000.0:
        unbound = False
        print('Failed at:', integrator.clock)
        break

In [ ]:
for i in range(800):
    print(integrator.getRateFromKey("b7->" + str(i+10)))
    #print(couplingMSM.getRate("b7->b" + str(i)))

In [ ]:
for i in range(10):
    print(couplingMSM.getRate("b7->b" + str(i)))